# <center> KITTIDATASET CLASS </center>

The following displays how the dataset is initialized and datapoints are visualized

In [ ]:
import os
os.chdir('..')

from data.kitti_dataset import KittiDataset

In [ ]:
kitti = KittiDataset() # initialize data object
print(kitti)

In [ ]:
# visualisation/ transformations check
idx = random.choice(range(kitti.num_files))

# while not any([_['name'] =='Truck' for _ in kitti.get_label(idx)]):
#   idx = random.choice(range(kitti.num_files))
  
print(idx)
plt.imshow(kitti.get_image(idx))
kitti.inspect_points(frame_idx=idx,downsample_voxel_size=0.05)

In [ ]:
# creates and load json file conatining cropped boxes
# kitti.save_cropped_boxes()
cropped_labels,cropped_cam_points = kitti.load_cropped_boxes()

In [ ]:
# visualizes isolated objects
kitti.vis_cropped_boxes(cropped_labels,cropped_cam_points,object_class='Pedestrian')

In [ ]:
kitti.vis_crop_aug_sampler()

In [ ]:
calib = kitti.get_calib(0)

In [ ]:
calib['cam_to_image']